# Tiền xử lý dữ liệu - Sentiment Analysis
Notebook này thực hiện tiền xử lý văn bản từ dataset IMDb, bao gồm làm sạch, tokenization, loại bỏ stopword, và vector hóa.

In [1]:
import pandas as pd

# Đọc dataset
df = pd.read_csv('../data/raw/IMDB Dataset.csv')

# Xem 5 dòng đầu
print("5 dòng đầu của dataset:")
display(df.head())

# Kiểm tra thông tin
print("\nThông tin dataset:")
print(df.info())

# Kiểm tra giá trị thiếu
print("\nGiá trị thiếu:")
print(df.isnull().sum())

5 dòng đầu của dataset:


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive



Thông tin dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB
None

Giá trị thiếu:
review       0
sentiment    0
dtype: int64


In [2]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Tải tài nguyên nltk
nltk.download('punkt')
nltk.download('stopwords')

# Hàm tiền xử lý văn bản
def preprocess_text(text):
    # Chuyển thành chữ thường
    text = text.lower()
    # Xóa HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Xóa ký tự đặc biệt, giữ chữ và số
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Tách từ (tokenization)
    tokens = word_tokenize(text)
    # Loại bỏ stopword
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Ghép lại thành chuỗi
    return ' '.join(tokens)

# Áp dụng hàm cho cột 'review'
df['cleaned_review'] = df['review'].apply(preprocess_text)

# Xem mẫu dữ liệu đã xử lý
print("\nMẫu dữ liệu sau khi làm sạch:")
display(df[['review', 'cleaned_review']].head())

[nltk_data] Downloading package punkt to /home/tankuvie/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tankuvie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Mẫu dữ liệu sau khi làm sạch:


,review,cleaned_review
0,One of the other reviewers has mentioned that ...,one reviewers mentioned watching 1 oz episode ...
1,A wonderful little production. <br /><br />The...,wonderful little production filming technique ...
2,I thought this was a wonderful way to spend ti...,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,basically theres family little boy jake thinks...
4,"Petter Mattei's ""Love in the Time of Money"" is...",petter matteis love time money visually stunni...


In [3]:
# Chuyển 'positive' thành 1, 'negative' thành 0
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# Kiểm tra
print("\nNhãn sau khi chuyển đổi:")
print(df['sentiment'].value_counts())


Nhãn sau khi chuyển đổi:
sentiment
1    25000
0    25000
Name: count, dtype: int64


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import joblib

# Khởi tạo TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)

# Vector hóa văn bản
X = vectorizer.fit_transform(df['cleaned_review'])
y = df['sentiment']

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Kiểm tra kích thước
print("\nKích thước dữ liệu:")
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")

# Lưu vectorizer
joblib.dump(vectorizer, '../models/tfidf_vectorizer.pkl')


Kích thước dữ liệu:
X_train: (40000, 5000), X_test: (10000, 5000)


['../models/tfidf_vectorizer.pkl']

In [5]:
# Lưu dữ liệu đã xử lý
df[['cleaned_review', 'sentiment']].to_csv('../data/processed/cleaned_reviews.csv', index=False)

# Kiểm tra file đã lưu
processed_df = pd.read_csv('../data/processed/cleaned_reviews.csv')
print("\nDữ liệu đã xử lý:")
display(processed_df.head())


Dữ liệu đã xử lý:


,cleaned_review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically theres family little boy jake thinks...,0
4,petter matteis love time money visually stunni...,1
